## Twitter API

In [1]:
from twython import Twython

TWITTER_APP_KEY = 'xxxxxx'  #supply the appropriate value
TWITTER_APP_KEY_SECRET = 'xxxxxx' 
TWITTER_ACCESS_TOKEN = 'xxxxxxx'
TWITTER_ACCESS_TOKEN_SECRET = 'xxxxxx'

In [ ]:
t = Twython(app_key=TWITTER_APP_KEY, 
            app_secret=TWITTER_APP_KEY_SECRET, 
            oauth_token=TWITTER_ACCESS_TOKEN, 
            oauth_token_secret=TWITTER_ACCESS_TOKEN_SECRET)

queries = ['#darkpattern', '#darkpatterns', '@darkpatterns']

search = t.search(q='#lol',   #**supply whatever query you want here**
                  count=100)

tweets = search['statuses']

for tweet in tweets:
  print tweet['id_str'], '\n', tweet['text'], '\n\n\n'

Link to Twitter API guide: https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

Link to StackOverflow for Twitter API: https://stackoverflow.com/questions/14156625/fetching-tweets-with-hashtag-from-twitter-using-python

## Reddit API

In [2]:
#! usr/bin/env python3
import praw
import pandas as pd
from datetime import datetime
import hashlib
import urllib.request
import os.path

import time
DELAY = 0.5
COMPUTER_USERNAME_HASH = "327529c973f4a2eca02eda86530afdacbcb2d04af59de078201a6c1d3e1f7af2"

In [3]:
### Input a file path to my Reddit credentials ###
# No peeking! Did you think I would put my computer's username in the code?
computer_username = input("Type in a computer username: ")
hashGen = hashlib.sha256()
hashGen.update(computer_username.encode('utf-8'))
if COMPUTER_USERNAME_HASH != hashGen.hexdigest(): 
    raise ValueError("Incorrect file path passed to code!")

Type in a computer username: openamiguel


In [4]:
### Access credentials at the given file ###
reddit_credentials_fpath = f"/Users/{computer_username}/Documents/REDDIT_API_CREDENTIALS.txt"
reddit_credentials_file = open(reddit_credentials_fpath, 'r')
reddit_credentials = reddit_credentials_file.read().split("\n")

In [5]:
### Access credentials from file on same machine ###
REDDIT_PERSONAL_USE_SCRIPT = reddit_credentials[0]
REDDIT_SECRET_KEY = reddit_credentials[1]
REDDIT_APP_NAME = reddit_credentials[2]
# No peeking! Did you think I would put my Reddit password in the code? 
# Username & password are unnecessary for public posts
REDDIT_USER_NAME = reddit_credentials[3]
REDDIT_LOGIN_PASSWORD = reddit_credentials[4]

In [6]:
### Plug credentials into a parser object ###
reddit = praw.Reddit(client_id=REDDIT_PERSONAL_USE_SCRIPT, \
                     client_secret=REDDIT_SECRET_KEY, \
                     user_agent=REDDIT_APP_NAME, \
                     username=REDDIT_USER_NAME, \
                     password=REDDIT_LOGIN_PASSWORD)

In [7]:
### Compute date & time from Unix timestamp ###
def get_time_from_unix(timestamp: int): 
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

In [8]:
### Defines the permissible flairs ###
ALLOWED_FLAIRS = [None, "Bad Unsubscribe Function", "Clickshaming", "Dark Pattern", \
                  "Bait and Switch", "Loaded Question"]
IMAGE_TAGS = ["i.redd.it", ".png", ".jpg", ".gif"]
VIDEO_TAGS = ["v.redd.it", ".mp4", ".mov"]

In [24]:
### Defines parsing parameters for a designated subreddit ###
subreddit = reddit.subreddit('assholedesign')
# Options: all, year, day, hour, week, month
top_subreddit = subreddit.top('day', limit=10)
mainpath = f"/Users/{computer_username}/Desktop/Reddit/"

data_rowslist = []
column_list = ["ID", "Title", "Media_Type", "Media_URL", "Date_Time", "Flair", \
               "Local_File_Name", "Is_OC", "Score", "Upvote_Ratio"]

In [25]:
### Parses a designated subreddit for certain data ###
for submission in top_subreddit: 
    # Removes all posts without URLs (i.e. posts without images)
    if submission.is_self: continue
    # Removes all posts without the proper flairs
    if submission.link_flair_text not in ALLOWED_FLAIRS: continue
    # Removes all posts with low score (might not be "asshole design")
    if submission.upvote_ratio < 0.6: continue
    # Parses date & time
    submission_datetime = get_time_from_unix(int(float(submission.created_utc)))
    # Decides if media is image or video
    submission_type = "Other"
    submission_url = submission.url
    if any([img_tag in submission_url for img_tag in IMAGE_TAGS]): submission_type = "Image"
    elif any([vid_tag in submission_url for vid_tag in VIDEO_TAGS]): submission_type = "Video"
    elif "imgur.com" in submission_url: submission_type = "Imgur_Non_Image"
    
    submission_localname = mainpath + submission.id + "." + submission_url.split('.')[-1]
    
    if submission_type == "Image" and not os.path.isfile(submission_localname): 
        urllib.request.urlretrieve(submission_url, submission_localname)
    else: submission_localname = "NA"
    
    rowdict = {}
    rowdict["ID"] = submission.id
    rowdict["Title"] = submission.title.replace('\t', '')
    rowdict["Media_Type"] = submission_type
    rowdict["Media_URL"] = submission_url
    rowdict["Date_Time"] = submission_datetime
    flair = submission.link_flair_text
    if flair != None: flair = flair.replace('\t', '')
    rowdict["Flair"] = flair
    rowdict["Local_File_Name"] = '/'.join(submission_localname.split('/')[3:])
    rowdict["Is_OC"] = submission.is_original_content
    rowdict["Score"] = submission.score
    rowdict["Upvote_Ratio"] = submission.upvote_ratio
    
    data_rowslist.append(rowdict)
    
    rowlist = [str(val) for val in rowdict.values()]
    print('\t'.join(rowlist))
    
    # print(submission.id)
    # print(submission.title)
    # time.sleep(DELAY)

fopsms	This shitty ad that compares you to a nazi	Image	https://i.redd.it/r8ngn4ttfto41.png	2020-03-25 12:55:05	Clickshaming		False	34793	0.89
fop1k1	I paid 20 dollars for this game.	Image	https://i.redd.it/ph0651bc5to41.png	2020-03-25 11:56:04	Bait and Switch		False	189	0.98
fouyy4	Wait a damn minute...	Image	https://i.redd.it/rt2oax1hzuo41.jpg	2020-03-25 18:04:46	Bait and Switch		False	115	0.98
fomhd5	So i now get Ad's on my 'Source' selection...	Image	https://i.redd.it/ye18u8ohzro41.jpg	2020-03-25 07:59:29	None		False	107	0.93
foztwk	Adding a weight to a laptop hard drive to make it seem bigger	Image	https://i.imgur.com/IbM1yYq.jpg	2020-03-25 22:37:29	None		False	134	0.89
foqpqg	Sending a letter that looks like information about the current pandemic solely to remind people rent is still due	Image	https://i.redd.it/u32dln7hrto41.jpg	2020-03-25 13:58:07	Bait and Switch		False	55	0.83
fou730	Shitty Scams like this during the whole COVID-19 pandemic	Image	https://i.redd.it/xevam06rruo41

In [126]:
### Save and print the data ###
data = pd.DataFrame(data_rowslist, columns=column_list)
print(data)
data.to_csv(mainpath + "DATA.txt", sep='\t', index=False)

       ID                                              Title Media_Type  \
0  fopsms         This shitty ad that compares you to a nazi      Image   
1  fop1k1                   I paid 20 dollars for this game.      Image   
2  fouyy4                              Wait a damn minute...      Image   
3  fomhd5      So i now get Ad's on my 'Source' selection...      Image   
4  foztwk  Adding a weight to a laptop hard drive to make...      Image   
5  foqpqg  Sending a letter that looks like information a...      Image   
6  fou730  Shitty Scams like this during the whole COVID-...      Image   
7  fovve0               I'm unable to shutdown my laptop now      Image   
8  fp34si       This mail coming from an eye doctor’s office      Image   

                             Media_URL            Date_Time            Flair  \
0  https://i.redd.it/r8ngn4ttfto41.png  2020-03-25 12:55:05     Clickshaming   
1  https://i.redd.it/ph0651bc5to41.png  2020-03-25 11:56:04  Bait and Switch   
2  https:

Link to Guide: https://www.storybench.org/how-to-scrape-reddit-with-python/